# HACETTEPE UNIVERSITY

## DEPARMENT OF COMPUTER ENGINEERING

### BBM 409 : Machine Learning Lab 2nd Assignment Report

# Part - I : Theory Questions

## 1st Question : MLE

 L(theta\x1,...,xn) = P(x1).P(x2)P(x3)....P(xn) = (2theta/3)^f0 + (1theta/3)^f1  (2(1-theta/3)^f2 ((1-theta/3))^f3 
                    = constant x theta^(f0+f1)(1-theta)^(f2+f3)
 
 
 (f0, f1, f2, f3 are respectively the frequencies of 0,1,2, and 3.)
 
 log L = constant + (f0 + f1)log(theta) - (f2 +f3)log(1-theta)
 
 dlogL / d(theta) = (f0 + f1)/theta + (f2 + f3)(1-theta) = 0
 
 (f0 + f1)log(1-theta) - (f2 +f3)log(theta) = 0
 
 (f0 + f1) = (f0 + f1 +f2 +f3)theta = n.theta
 
 theta = (f0 +f1)/n
 
 n is the number of observations and f0 and f1 are respectively the number of 0’s and number of 1’s
 
 n this sample, f0 = 2, f1 =3, f2 = 3, f3 = 2, n = 10
 
 (f0 +f1)/n = (2+3)/10 = 0.5
 
 


# Part - II : Detection of Fake News

In this part , the code implements the Naive Bayes algorithm with using Unigram and Bigram features as BoW methods

The code reads the trainSet and testSet . Creating Features it detects the News , fake or real .

In [7]:
import pandas as pd
from nltk import ngrams
import math
import collections
import math
from sklearn.feature_extraction import text

In [2]:


def readFiles():

    list_real = []
    list_fake = []
    list_test = []
    with open('⁨⁨data⁩/clean_fake-Train.txt', 'r+') as f:
        for line in f.readlines():
            list_fake.append(line)
    with open('data/clean_real-Train.txt', 'r+') as f:
        for line in f.readlines():
            list_real.append(line)

    testFrame = pd.read_csv('data/test.csv')
    testFrame.columns = ['Id', 'Category']
    list_test = testFrame['Id'].tolist()
    return list_real, list_fake,list_test, testFrame

The code read the files from data folder and create the lists and dataFrame to hold the data 

In [3]:
def findCount(word, dict):
    for key in dict.keys():
        if word == key:
            return dict[key][0]
        else:
            return 0

 The function search on the dictionary and if the key is matched , it returns the count of key

In [4]:
def findKeyWords(list):
    words = []
    for i in range(len(list)):

        array = list[i].split()

        for j in range(len(array)):
            words.append(array[j])
    frequencyOfWords = collections.Counter()
    frequencyOfWords.update(words)

    print(frequencyOfWords.most_common(5)[2:5])
    return frequencyOfWords

findkeyWords function count the duplicate elements and convert to dictionary with elements of list

In [ ]:
def addUnseenWords(test,real,fake):

    for key in test.keys():
        if not key in real.keys():
            real[key] = 1
        if not key in fake.keys():
            fake[key] = 1
    return real, fake

addUnseenWords apply the Laplace smoothing if any unseen items are in the trainSet(fake or real), the code adds to unigram model and frequency of these words is 1

In [5]:
def buildUnigram(frequency, ngramDict):
    totalCount = sum(frequency.values());
    for key, value in frequency.items():
        # Laplace Smoothing #
        ngramDict[key] = [(value + 1), math.log(value / (totalCount + len(frequency.keys())))]
    return ngramDict

buildUnigram function implements the unigram feature according to values and probablities of  the elements in trainSet

In [6]:
def buildBigram(list, grams, bigramDict):
    bigram = []
    for i in range(len(list)):
        array = list[i].split()
        bigram.append(ngrams(array, 2))
    for i in range(len(bigram)):
        for gram in bigram[i]:
            grams.append(gram)
    frequencyOfBiGram = collections.Counter()
    frequencyOfBiGram.update(grams)
    for key, value in frequencyOfBiGram.items():
        bigramDict[key] = value
    return bigramDict

buildBigram function implements the bigram feature with using frequency of the words . The code uses the ngram function
of nltk library to create the bigram models . collection.counter produces  unique elements with counts and these items
are held in the bigram Dictionary

In [1]:
def calculateProbablity(testNgram, realNgram, fakeNgram, probReal, probFake):
    for grams in testNgram:
        grams = ''.join(grams)
        if grams in realNgram.keys():
            probReal = probReal + realNgram[grams][1]

        if grams in fakeNgram.keys():
            probFake = probFake + fakeNgram[grams][1]

    if probReal == max(probFake, probReal):
        return "real"
    else:
        return "fake"

calculate Probability function calculates the probablities of testSet elements according to unigram model and check the minimum value to compare testSet categories . The code calculate with using logarithmic function so all probabilities are added and first value of probReal and probFake equal to probablity of category in trainSet elements . It returns the label function to find the accuracy . 

In [8]:
def newsLabelUni(testFrame, frequencyReal, frequencyFake, unigramDictReal, unigramDictFake, probReal, probFake):
    right_prediction = 0
    for i in range(len(testFrame.Id)):
        testUnigram = ngrams(testFrame.Id[i].split(), 1)

        if calculateProbablity(testUnigram, frequencyReal, frequencyFake, unigramDictReal, unigramDictFake, probReal,
                               probFake) == \
                testFrame.Category[i]:
            right_prediction = right_prediction + 1
    print("Accuracy of the Unigram Model : " + str(round(100 * (right_prediction / len(testFrame.Id)))) + "%")

newsLabelUni function calculate the accuracy of the unigram model and check the categories and predictions of the code.


In [3]:
def calculateProbBigram(testNgram, frequencyReal, frequencyFake, uniReal, uniFake, realNgram, fakeNgram, probReal,
                        probFake):
    deliminator = len(frequencyReal)
    deliminatorFake = len(frequencyFake)

    for grams in testNgram:

        if grams in realNgram.keys():
            probReal = probReal + math.log((realNgram[grams] + 1) / (findCount(grams[0], uniReal) + deliminator))
        else:

                probReal = probReal + math.log(1 / (findCount(grams[0], uniReal) + deliminator))

        if grams in fakeNgram.keys():
            probFake = probFake + math.log((fakeNgram[grams] + 1) / (findCount(grams[0], uniFake) + deliminatorFake))
        else:
                probFake = probFake + math.log(1 / (deliminatorFake + findCount(grams[0], uniFake)))

    if probReal == max(probFake, probReal):
        return "real"
    else:
        return "fake"

 The calculateProbBigram function labels the testSet element with using bigram feature , probabilities is calculated
logarithmic function and picks the bigger one because all probabilities are between 0 and 1 , so they were negative 
expressions .

In [4]:
def newsLabelBi(testFrame, frequencyReal, frequencyFake, unigramDictReal, unigramDictFake, bigramDictReal,
                bigramDictFake, probReal, probFake):
    right_prediction = 0
    realWords = []
    fakeWords = []

    for i in range(len(testFrame.Id)):
        testFrame.Id[i] = "<s> " + testFrame.Id[i] + " <n>"
        words = testFrame.Id[i].split()
        testBigram = ngrams(testFrame.Id[i].split(), 2)
        label = calculateProbBigram(testBigram, frequencyReal, frequencyFake, unigramDictReal, unigramDictFake,
                                    bigramDictReal, bigramDictFake, probReal, probFake)

        if label == testFrame.Category[i]:
            right_prediction = right_prediction + 1

        if label == "real":
            for i in range(len(words)):
                realWords.append(words[i])
        else:
            for i in range(len(words)):
                fakeWords.append(words[i])

    print("Accuracy of the Bigram Model  : " + str(round(100 * (right_prediction / len(testFrame.Id)))) + "%")
    listWords(realWords, fakeWords, unigramDictReal, unigramDictFake)

newsLabelBi function calculate the accuracy of the bigram model and check the categories and predictions of the code.



In [10]:
def listWords(realWords, fakeWords, dictReal, dictFake):
    absentReal = {}
    absentFake = {}

    for key,value in dictReal.items():
        if not key in realWords:
            absentReal[key] = value[0]

    for key,value in dictFake.items():
        if not key in fakeWords:
            absentFake[key] = value[0]

    frequencyOfReals = collections.Counter()
    frequencyOfReals.update(realWords)

    frequencyOfFakes = collections.Counter()
    frequencyOfFakes.update(fakeWords)
    # The code sorts the dictionary according to its value #
    sortedAbsentReal = [(k, absentReal[k]) for k in sorted(absentReal, key=absentReal.get, reverse=True)]
    sortedAbsentFake = [(k, absentFake[k]) for k in sorted(absentFake, key=absentFake.get, reverse=True)]

    print("List the 10 words whose presence most strongly predicts that the news is real :")
    print(frequencyOfReals.most_common(10))
    print("List the 10 words whose presence most strongly predicts that the news is fake :")
    print(frequencyOfFakes.most_common(10))
    print("List the 10 words whose absence most strongly predicts that the news is real :")
    print(sortedAbsentReal[:10])
    print("List the 10 words whose absence most strongly predicts that the news is fake :")
    print(sortedAbsentFake[:10])

    # Effects of Stop Words #
    add = ["<s>", "<n>"]
    stop_words = text.ENGLISH_STOP_WORDS.union(add)
    filtered_realNews = {}
    filtered_fakeNews = {}

    for key, value in frequencyOfReals.items():
        if key not in stop_words:
            filtered_realNews[key] = value

    for key, value in frequencyOfFakes.items():
        if key not in stop_words:
            filtered_fakeNews[key] = value

    filteredReals = collections.Counter()
    filteredReals.update(filtered_realNews)

    filteredFakes = collections.Counter()
    filteredFakes.update(filtered_fakeNews)
    print("List the 10 non-stopwords that most strongly predict that the news is real :")
    print(filteredReals.most_common(10))
    print("List the 10 non-stopwords that most strongly predict that the news is fake :")
    print(filteredFakes.most_common(10))

The listWords function creates two lists and holds the elements of testsets as real labeled or fake labeled . 
After that , the code find the words which are in trainSet but not in realNewsLabel . Same method is applied for realFakeLabel and to find most strongly 10 words for prediction , frequency of words are used . Counter function of collection library is used to count words .

###  Effects of StopWords

Classification refers to compare the textfile to compare the each other which is used for prediction of test elements.
Context of train sets include many meanless word and these may  confuse the calculation and we can use the resources effectively on the other hand simple words may have a big importance to understand the test elements . For example : 
" good " or " not good" are the similar expression but their means are opposite . That's situation is the make  a conflict for prediction .
As we see the results , Stop words produce the better solution to find the words which have bigger probability for prediction . Because this project based on binary classification and it is not important meaning of texts . However we  assume that the code implements a sentiment classfication , probably stopwords produce the wrong results . It depends on our mission .

In [ ]:
# initialization of dictionaries to hold the N-Gram Models (Unigram and Bigram)
unigramDictReal = {}
unigramDictFake = {}
unigramDictTest ={}
bigramDictReal = {}
bigramDictFake = {}
gramsReal = []
gramsFake = []

list_real, list_fake,list_test, testFrame = rd.readFiles()


# add a specific expression to detect head of the sentence  #
for i in range(len(list_real)):
    list_real[i] = list_real[i][0:-2]
    list_real[i] = "<s> " + list_real[i]+" <n>"
for j in range(len(list_fake)):
    list_fake[j] = list_fake[j][0:-2]
    list_fake[j] = "<s> "+ list_fake[j]+" <n>"

# Most commonly keywords #
print("Most commonly 3 keywords for Real News:")
frequencyReal = findKeyWords(list_real)
print(frequencyReal.most_common(5)[2:5])
print("Most commonly 3 keywords for Fake News:")
frequencyFake = findKeyWords(list_fake)
print(frequencyFake.most_common(5)[2:5])

frequencyTest = findKeyWords(list_test)
frequencyReal, frequencyFake = ng.addUnseenWords(frequencyTest, frequencyReal, frequencyFake)
unigramDictReal = buildUnigram(frequencyReal, unigramDictReal)
unigramDictFake = buildUnigram(frequencyFake, unigramDictFake)

bigramDictReal = buildBigram(list_real, gramsReal, bigramDictReal)
bigramDictFake = buildBigram(list_fake, gramsFake, bigramDictFake)

probReal = len(list_real) / (len(list_real)+len(list_fake))
probFake = len(list_fake) / (len(list_real)+len(list_fake))

newsLabelUni(testFrame, unigramDictReal, unigramDictFake
                , math.log(probReal), math.log(probFake))

newsLabelBi(testFrame, frequencyReal, frequencyFake,unigramDictReal,unigramDictFake, bigramDictReal, bigramDictFake
                , math.log(probReal), math.log(probFake))

The code starts from this script . To detect the head of lines and the end of lines the code adds the special expressions . For head of line :  "s" and for end of line : "n" are added before create the unigram and bigram models . probsReal and probsFake variables hold the probability of their categories according to counts of the lines .

## Conclusion

### Unigram Model

One of its characteristics is that it doesn't take the ordering of the words into account, so the order doesn't make a difference in how words are tagged or split up.

### Bigram Model

Bigram Models, on the other hand do care about the order of the words, so it considers the context of each word by analyzing it by pairs. Whereas a unigram model will tag a word independent of the other words .

 In many instances the order of the words might not matter at all. But sometimes meaning is very important to detect the category of the test elements so the bigram model is more successful approach than unigram model .
 